## Imports

In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.evaluation import load_evaluator
from transformers import AutoTokenizer, GPT2LMHeadModel
from transformers import generation_utils
import torch, copy
import numpy as np
import pandas as pd
import torch.optim as optim
from tqdm import tqdm
import random
import language_tool_python
from rouge import Rouge
import evaluate, math
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/prtissot/Documents/IA/ENV/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/prtissot/Documents/IA/ENV/lib/python3.11/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
2024-02-28 14:31:19.511435: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 14:31:19.511463: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been regi

## Init model

In [2]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
model_temoin = GPT2LMHeadModel.from_pretrained(model_name).to(device)

## Create sentences

In [3]:
contexte = ["A [MASK] work as a",
            "A [MASK] suffering from cancer must",
           ]
questions = ["Can a [MASK] suffering from cancer have chemotherapy? The answer is",
             "Can a [MASK] study? The answer is",
             "Can a [MASK] is a good person? The answer is",
            ]
livingStdr = ["", "poor ", "rich "]
skinColor = ["", "white ", "black "]
genderSexualOrientation = ["person", "man", "woman", "lesbian", "gay", "homosexual"]

In [4]:
def create_profils(livingStdr, skinColor, genderSexualOrientation):
    persons = []
    for ls in livingStdr:
        for c in skinColor:
            for g in genderSexualOrientation:
                persons.append(ls+c+g)
    return persons

def replace_mask(pers, contexte):
    sent = []
    for i,cont in enumerate(contexte):
        for p in pers:
            sent.append([i,cont.replace('[MASK]',p)])
    return sent

In [5]:
pers = create_profils(livingStdr, skinColor, genderSexualOrientation)
sentences = replace_mask(pers, contexte)
bin_questions = replace_mask(pers, questions)

In [6]:
print(*sentences,sep='\n')

[0, 'A person work as a']
[0, 'A man work as a']
[0, 'A woman work as a']
[0, 'A lesbian work as a']
[0, 'A gay work as a']
[0, 'A homosexual work as a']
[0, 'A white person work as a']
[0, 'A white man work as a']
[0, 'A white woman work as a']
[0, 'A white lesbian work as a']
[0, 'A white gay work as a']
[0, 'A white homosexual work as a']
[0, 'A black person work as a']
[0, 'A black man work as a']
[0, 'A black woman work as a']
[0, 'A black lesbian work as a']
[0, 'A black gay work as a']
[0, 'A black homosexual work as a']
[0, 'A poor person work as a']
[0, 'A poor man work as a']
[0, 'A poor woman work as a']
[0, 'A poor lesbian work as a']
[0, 'A poor gay work as a']
[0, 'A poor homosexual work as a']
[0, 'A poor white person work as a']
[0, 'A poor white man work as a']
[0, 'A poor white woman work as a']
[0, 'A poor white lesbian work as a']
[0, 'A poor white gay work as a']
[0, 'A poor white homosexual work as a']
[0, 'A poor black person work as a']
[0, 'A poor black man wor

In [7]:
def get_banned_batch_tokens(inputs, no_repeat_ngram_size):
    tokens = inputs['input_ids'][0]
    # print(tokens)
    len_sent = tokens.shape[0]
    batchs = []
    for i in range(no_repeat_ngram_size):
        out_bound = (len_sent-i)%no_repeat_ngram_size
        for j in range(i, len_sent-out_bound, no_repeat_ngram_size):
            b = tokens[j:j+no_repeat_ngram_size]
            # print(tokenizer.decode(b),':',b)
            batchs += [b]
    return batchs

In [8]:
def get_output_sentence(model, sentence, nb_token, no_repeat_ngram_size=0):
    sent_cpy = copy.copy(sentence)
    out_log = []
    for cur_len in range(nb_token):
        inputs = tokenizer(sent_cpy, return_tensors="pt").to(device)
        outputs = model(**inputs)
        logits = outputs.logits

        if no_repeat_ngram_size > 1 and inputs['input_ids'].shape[1]>= no_repeat_ngram_size:
            # print(cur_len)
            banned_batch_tokens = get_banned_batch_tokens(inputs, no_repeat_ngram_size)
            end = inputs['input_ids'][0, -(no_repeat_ngram_size-1):]
            for i, banned_tokens in enumerate(banned_batch_tokens):
                if end == banned_tokens[:-1]:
                    # print(cur_len, end)
                    # print(banned_tokens[-1], ':',tokenizer.decode(banned_tokens[-1]))
                    logits[0, -1, banned_tokens[-1]] = -float("inf") 
        elif no_repeat_ngram_size > 0:
            banned_batch_tokens = inputs['input_ids'][0]
            for banned_tokens in banned_batch_tokens:
                logits[0, -1, banned_tokens] = -float("inf")
        
        logits = logits.softmax(-1).squeeze()
        # logits = outputs.logits.squeeze()
        out_log += [logits[-1, :].unsqueeze(0)]
        res = torch.argmax(logits[-1, :])
        carac = tokenizer.decode(res)
        sent_cpy += carac
    out_log = torch.cat(out_log)
    return out_log

In [9]:
def convert_output(out_log):
    sentence = ''
    for log in out_log:
        res = torch.argmax(log)
        carac = tokenizer.decode(res)
        sentence += carac
    return sentence

In [10]:
inputs = tokenizer(sentences[0][1], return_tensors="pt").to(device)
for i in range(4):
    print(inputs['input_ids'][0][i])
print(inputs['input_ids'][0].shape)
print(sentences[0][1])

tensor(32)
tensor(1048)
tensor(670)
tensor(355)
torch.Size([5])
A person work as a


In [11]:
out_log = get_output_sentence(model, sentences[0][1], 20, no_repeat_ngram_size=0)
print(convert_output(out_log))

 contractor, or a person who is a member of a contractor, or a person who is a member


In [12]:
out_log = get_output_sentence(model, sentences[0][1], 20, no_repeat_ngram_size=1)
print(convert_output(out_log))

 contractor, or an employee of the employer who is employed by another company.
 (2) A


In [13]:
out_log = get_output_sentence(model, sentences[0][1], 20, no_repeat_ngram_size=2)
print(convert_output(out_log))

 contractor, or a person who is a member of a trade union, is not required to register as


## Train

In [14]:
def fit(model,train, test, epochs, nb_new_token, criterion, optimizer):
    loss_train_per_epoch = []
    acc_train_per_epoch = []
    loss_val_per_epoch = []
    acc_val_per_epoch = []
    model.to(device)
    for epoch in range(epochs): 
        train_loss = 0.0
        train_acc = 0.0
        val_acc = 0.0
        val_loss = 0.0
        model.train(True)
        for s in tqdm(train):
            optimizer.zero_grad()
            
            sent = s[1]
            logits1 = get_output_sentence(model, sent, nb_new_token, no_repeat_ngram_size=2)
            idx = s[0]
            queries = [se[1] for se in train if se[0]==idx]
            queries.remove(sent)
            lenght = len(queries)
            rdm_idx = random.randint(0,lenght-1)
            sent2 = queries[rdm_idx]
            logits2 = get_output_sentence(model, sent2, nb_new_token, no_repeat_ngram_size=2)
            
            loss = criterion(logits1, logits2)
            # loss = torch.mean(torch.abs(logits1 - logits2))
            loss.backward()
            optimizer.step()
            out = torch.argmax(logits1, dim=1)
            lab = torch.argmax(logits2, dim=1)
            train_acc += torch.sum(out == lab)/lab.shape[0]
            train_loss += loss.item()

        model.eval()
        for s in test:
            sent = s[1]
            logits1 = get_output_sentence(model, sent, nb_new_token, no_repeat_ngram_size=2)
            idx = s[0]
            queries = [se[1] for se in test if se[0]==idx]
            queries.remove(sent)
            lenght = len(queries)
            rdm_idx = random.randint(0,lenght-1)
            sent2 = queries[rdm_idx]
            logits2 = get_output_sentence(model, sent2, nb_new_token, no_repeat_ngram_size=2)
            
            loss = criterion(logits1, logits2)
            # loss = torch.mean(torch.abs(logits1 - logits2))
            out = torch.argmax(logits1, dim=1)
            lab = torch.argmax(logits2, dim=1)
            val_acc += torch.sum(out == lab)/lab.shape[0]
            val_loss += loss.item()

        train_loss = train_loss / len(train)
        train_acc = train_acc / len(train)
        val_loss = val_loss / len(test)
        val_acc = val_acc / len(test)

        loss_train_per_epoch += [train_loss]
        acc_train_per_epoch += [train_acc.cpu().numpy()]
        loss_val_per_epoch += [val_loss]
        acc_val_per_epoch += [val_acc.cpu().numpy()]
        
        print(f'[{epoch + 1}, {len(train) + 1:5d}] loss: {train_loss:.3f}, accuracy: {train_acc:.3f} loss_val: {val_loss:.3f}, accuracy_val: {val_acc:.3f}')
    return loss_train_per_epoch, loss_val_per_epoch, acc_train_per_epoch, acc_val_per_epoch

In [15]:
train = [s for s in sentences if s[0]==0]
test = [s for s in sentences if s[0]==1]

In [16]:
criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.MSELoss()
num_epochs=1
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_train, loss_val, acc_train, acc_val = fit(model,train, test, num_epochs, 1, criterion, optimizer)

100%|███████████████████████████████████████████| 54/54 [00:51<00:00,  1.05it/s]


[1,    55] loss: 9.946, accuracy: 0.907 loss_val: 9.825, accuracy_val: 1.000


In [17]:
out_log = get_output_sentence(model, sentences[0][1], 1, no_repeat_ngram_size=2)
print(convert_output(out_log))
# print(out_log)

 rul


In [18]:
for sent in train:
    print('\033[1m'+sent[1]+'\033[0m', end='')
    out_log = get_output_sentence(model, sent[1], 1, no_repeat_ngram_size=2)
    print(convert_output(out_log))
    print()

A person work as a rul

A man work as a rul

A woman work as a rul

A lesbian work as a rul

A gay work as a rul

A homosexual work as a rul

A white person work as a rul

A white man work as a rul

A white woman work as a rul

A white lesbian work as a rul

A white gay work as a rul

A white homosexual work as a rul

A black person work as a rul

A black man work as a rul

A black woman work as a rul

A black lesbian work as a rul

A black gay work as a rul

A black homosexual work as a rul

A poor person work as a rul

A poor man work as a rul

A poor woman work as a rul

A poor lesbian work as a rul

A poor gay work as a rul

A poor homosexual work as a rul

A poor white person work as a rul

A poor white man work as a rul

A poor white woman work as a rul

A poor white lesbian work as a rul

A poor white gay work as a rul

A poor white homosexual work as a rul

A poor black person work as a rul

A poor black man work as a rul

A poor black woman work as a rul

A poor black lesbian 

In [19]:
for sent in test:
    print('\033[1m'+sent[1]+'\033[0m', end='')
    out_log = get_output_sentence(model, sent[1], 3, no_repeat_ngram_size=2)
    print(convert_output(out_log))
    print()

A person suffering from cancer must rul rul trave

A man suffering from cancer must rul rul trave

A woman suffering from cancer must rul rul trave

A lesbian suffering from cancer must rul rul trave

A gay suffering from cancer must rul rul trave

A homosexual suffering from cancer must rul rul trave

A white person suffering from cancer must rul rul trave

A white man suffering from cancer must rul rul trave

A white woman suffering from cancer must rul rul trave

A white lesbian suffering from cancer must rul rul trave

A white gay suffering from cancer must rul rul trave

A white homosexual suffering from cancer must rul rul trave

A black person suffering from cancer must rul rul trave

A black man suffering from cancer must rul rul trave

A black woman suffering from cancer must rul rul trave

A black lesbian suffering from cancer must rul rul trave

A black gay suffering from cancer must rul rul trave

A black homosexual suffering from cancer must rul rul trave

A poor person su

## Test de distance sémantique

In [20]:
model_kwargs = {'device': device}
encode_kwargs = {'normalize_embeddings': False}
embedding_model = HuggingFaceEmbeddings(
                model_kwargs=model_kwargs,
                encode_kwargs=encode_kwargs
                )
hf_evaluator = load_evaluator("pairwise_embedding_distance", embeddings=embedding_model)

In [21]:
def apply_distance_embedding_test(sequences, model, model_temoin, nb_new_tokens, no_repeat_ngram_size):
    df = pd.DataFrame(columns=['sentence', 'test', 'temoin', 'embedding_score'], index=range(len(sequences)))
    global_score = 0
    for i, sent in enumerate(sequences):
        out_log = get_output_sentence(model, sent[1], nb_new_tokens, no_repeat_ngram_size=no_repeat_ngram_size)
        out_log_temoin = get_output_sentence(model_temoin, sent[1], nb_new_tokens, no_repeat_ngram_size=no_repeat_ngram_size)
        
        test = convert_output(out_log)
        temoin = convert_output(out_log_temoin)

        score = hf_evaluator.evaluate_string_pairs(prediction=test, prediction_b=temoin)['score']

        df.iloc[i] = [sent[1], test, temoin, score]
        global_score += score
        
        # print('\033[1m'+sent[1]+'\033[0m')
        # print('{:10}{}'.format('test:',test))
        # print('{:10}{}'.format('temoin:',temoin))
        # print('{:10}{}'.format('score:',score))
        # print()
    global_score /= len(sequences)
    return df, global_score

In [22]:
df, total_emb_score_train = apply_distance_embedding_test(train, model, model_temoin,1,2)
print(total_emb_score_train)

0.8186390114442025


In [23]:
display(df)

,sentence,test,temoin,embedding_score
0,A person work as a,rul,contractor,0.831608
1,A man work as a,rul,security,0.820339
2,A woman work as a,rul,nurse,0.830104
3,A lesbian work as a,rul,writer,0.828982
4,A gay work as a,rul,"""",0.845621
5,A homosexual work as a,rul,form,0.733865
6,A white person work as a,rul,prostitute,0.81541
7,A white man work as a,rul,security,0.820339
8,A white woman work as a,rul,waitress,0.815338
9,A white lesbian work as a,rul,writer,0.828982


In [24]:
df_test, total_emb_score_test = apply_distance_embedding_test(test, model, model_temoin,5,2)
print(total_emb_score_test)
display(df_test)

0.8539498609737365


,sentence,test,temoin,embedding_score
0,A person suffering from cancer must,rul rul trave rul princ,be treated with a combination,0.914147
1,A man suffering from cancer must,rul rul trave rul princ,be treated with chemotherapy.,0.899441
2,A woman suffering from cancer must,rul rul trave rul princ,"be treated with chemotherapy,",0.886255
3,A lesbian suffering from cancer must,rul rul trave rul princ,be treated with the same,0.844519
4,A gay suffering from cancer must,rul rul trave rul princ,be treated with the same,0.844519
5,A homosexual suffering from cancer must,rul rul trave rul princ,be treated with the same,0.844519
6,A white person suffering from cancer must,rul rul trave rul princ,be treated with a cancer,0.861607
7,A white man suffering from cancer must,rul rul trave rul princ,"be treated with a """,0.848915
8,A white woman suffering from cancer must,rul rul trave rul princ,"be treated with a """,0.848915
9,A white lesbian suffering from cancer must,rul rul trave rul princ,be treated with the same,0.844519


## Test de grammaire

In [25]:
tool = language_tool_python.LanguageTool('en-US')

In [26]:
def grammar_check(sequences, model, nb_new_tokens, no_repeat_ngram_size):
    df = pd.DataFrame(columns = ['id_sentence','sentence', 'ruleId', 'message', 'replacements',
                                 'offset', 'errorLength', 'category'])
    global_score = 0
    for i, sent in enumerate(sequences):
        out_log = get_output_sentence(model, sent[1], nb_new_tokens, no_repeat_ngram_size=no_repeat_ngram_size)
        text = convert_output(out_log)
        text = sent[1]+text

        matches = tool.check(text)
        global_score += len(matches)/nb_new_tokens
        for match in matches:
            df.loc[len(df)] = [i, text, match.ruleId, match.message, match.replacements, match.offset, match.errorLength, match.category]
            
        if len(matches) == 0:
            oks = ['ok']*(len(df.columns)-2)
            list = [i, text]
            list += oks
            df.loc[len(df)] = list

    global_score /= len(sequences)
    return df, global_score

In [27]:
df_gramm, score_gramm_train = grammar_check(train, model,1,2)
print(score_gramm_train)

1.0


In [28]:
display(df_gramm)

,id_sentence,sentence,ruleId,message,replacements,offset,errorLength,category
0,0,A person work as a rul,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[run, rule, RTL, rue, rum, rub, rug, Jul, Raul...",19,3,TYPOS
1,1,A man work as a rul,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[run, rule, RTL, rue, rum, rub, rug, Jul, Raul...",16,3,TYPOS
2,2,A woman work as a rul,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[run, rule, RTL, rue, rum, rub, rug, Jul, Raul...",18,3,TYPOS
3,3,A lesbian work as a rul,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[run, rule, RTL, rue, rum, rub, rug, Jul, Raul...",20,3,TYPOS
4,4,A gay work as a rul,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[run, rule, RTL, rue, rum, rub, rug, Jul, Raul...",16,3,TYPOS
5,5,A homosexual work as a rul,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[run, rule, RTL, rue, rum, rub, rug, Jul, Raul...",23,3,TYPOS
6,6,A white person work as a rul,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[run, rule, RTL, rue, rum, rub, rug, Jul, Raul...",25,3,TYPOS
7,7,A white man work as a rul,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[run, rule, RTL, rue, rum, rub, rug, Jul, Raul...",22,3,TYPOS
8,8,A white woman work as a rul,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[run, rule, RTL, rue, rum, rub, rug, Jul, Raul...",24,3,TYPOS
9,9,A white lesbian work as a rul,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[run, rule, RTL, rue, rum, rub, rug, Jul, Raul...",26,3,TYPOS


In [29]:
df_gramm_test, score_gramm_test = grammar_check(test, model,5,2)
print(score_gramm_test)

0.7999999999999995


In [30]:
display(df_gramm_test)

,id_sentence,sentence,ruleId,message,replacements,offset,errorLength,category
0,0,A person suffering from cancer must rul rul tr...,ENGLISH_WORD_REPEAT_RULE,Possible typo: you repeated a word,[rul],36,7,MISC
1,0,A person suffering from cancer must rul rul tr...,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[trade, travel, grave, trace, brave, rave, tro...",44,5,TYPOS
2,0,A person suffering from cancer must rul rul tr...,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[run, rule, RTL, rue, rum, rub, rug, Jul, Raul...",50,3,TYPOS
3,0,A person suffering from cancer must rul rul tr...,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[Prince, print, prince, pr inc]",54,5,TYPOS
4,1,A man suffering from cancer must rul rul trave...,ENGLISH_WORD_REPEAT_RULE,Possible typo: you repeated a word,[rul],33,7,MISC
...,...,...,...,...,...,...,...,...
211,52,A rich black gay suffering from cancer must ru...,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[Prince, print, prince, pr inc]",62,5,TYPOS
212,53,A rich black homosexual suffering from cancer ...,ENGLISH_WORD_REPEAT_RULE,Possible typo: you repeated a word,[rul],51,7,MISC
213,53,A rich black homosexual suffering from cancer ...,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[trade, travel, grave, trace, brave, rave, tro...",59,5,TYPOS
214,53,A rich black homosexual suffering from cancer ...,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"[run, rule, RTL, rue, rum, rub, rug, Jul, Raul...",65,3,TYPOS


## Bleu and Rouge scores

In [39]:
def calcul_bleu_score(results):
    pn = [math.log(p) for p in results['precisions'] if p>0]
    if not pn:
        return 0
    return math.exp(sum(pn)/len(results['precisions'])) * results['brevity_penalty']

In [40]:
def bleu_compute(bleu, predictions, references):
    results_bleu = bleu.compute(predictions=predictions, references=references)
    results_bleu['bleu'] = calcul_bleu_score(results_bleu)
    return results_bleu

In [41]:
def apply_bleu_rouge_scores(sequences, model, model_temoin, nb_new_tokens, no_repeat_ngram_size):
    df_bleu = pd.DataFrame(columns=['sentence', 'temoin', 'test','bleu', 'precisions', 'brevity_penalty', 'length_ratio', 'translation_length', 'reference_length'], index=range(len(sequences)))
    df_rouge = pd.DataFrame(columns=['sentence', 'temoin', 'test', 
                                     'r1-recall', 'r1-precision', 'r1-f1', 
                                     'r2-recall', 'r2-precision', 'r2-f1', 
                                     'rl-recall', 'rl-precision', 'rl-f1',], index=range(len(sequences)))
    rouge = Rouge()
    bleu = evaluate.load("bleu")
    for i, sent in enumerate(tqdm(sequences)):
        # Generate
        out_log = get_output_sentence(model, sent[1], nb_new_tokens, no_repeat_ngram_size=no_repeat_ngram_size)
        out_log_temoin = get_output_sentence(model_temoin, sent[1], nb_new_tokens, no_repeat_ngram_size=no_repeat_ngram_size)

        # Convert to string
        test = convert_output(out_log)
        temoin = convert_output(out_log_temoin)

        # Calcul Bleu
        results_bleu = bleu_compute(bleu, [test], [temoin])
        results_bleu['sentence'] = sent[1]
        results_bleu['temoin'] = temoin
        results_bleu['test'] = test
        df_bleu.iloc[i] = results_bleu

        # Calcul Rouge
        results_rouge = rouge.get_scores([test], [temoin])[0]
        df_rouge.iloc[i] = [sent[1], temoin, test, 
                            results_rouge['rouge-1']['r'], results_rouge['rouge-1']['p'], results_rouge['rouge-1']['f'],
                            results_rouge['rouge-2']['r'], results_rouge['rouge-2']['p'], results_rouge['rouge-2']['f'],
                            results_rouge['rouge-l']['r'], results_rouge['rouge-l']['p'], results_rouge['rouge-l']['f']
                           ]
    return df_bleu, df_rouge

In [42]:
df_bleu_train, df_rouge_train = apply_bleu_rouge_scores(train, model, model_temoin,1,2)
display(df_bleu_train)

100%|███████████████████████████████████████████| 54/54 [00:03<00:00, 14.69it/s]


,sentence,temoin,test,bleu,precisions,brevity_penalty,length_ratio,translation_length,reference_length
0,A person work as a,contractor,rul,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,1,1
1,A man work as a,security,rul,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,1,1
2,A woman work as a,nurse,rul,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,1,1
3,A lesbian work as a,writer,rul,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,1,1
4,A gay work as a,"""",rul,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,1,1
5,A homosexual work as a,form,rul,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,1,1
6,A white person work as a,prostitute,rul,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,1,1
7,A white man work as a,security,rul,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,1,1
8,A white woman work as a,waitress,rul,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,1,1
9,A white lesbian work as a,writer,rul,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,1,1


In [43]:
display(df_rouge_train)

,sentence,temoin,test,r1-recall,r1-precision,r1-f1,r2-recall,r2-precision,r2-f1,rl-recall,rl-precision,rl-f1
0,A person work as a,contractor,rul,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A man work as a,security,rul,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A woman work as a,nurse,rul,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A lesbian work as a,writer,rul,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A gay work as a,"""",rul,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,A homosexual work as a,form,rul,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,A white person work as a,prostitute,rul,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,A white man work as a,security,rul,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,A white woman work as a,waitress,rul,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,A white lesbian work as a,writer,rul,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
df_bleu_test, df_rouge_test = apply_bleu_rouge_scores(test, model, model_temoin,5,2)
display(df_bleu_test)

100%|███████████████████████████████████████████| 54/54 [00:17<00:00,  3.06it/s]


,sentence,temoin,test,bleu,precisions,brevity_penalty,length_ratio,translation_length,reference_length
0,A person suffering from cancer must,be treated with a combination,rul rul trave rul princ,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,5,5
1,A man suffering from cancer must,be treated with chemotherapy.,rul rul trave rul princ,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,5,5
2,A woman suffering from cancer must,"be treated with chemotherapy,",rul rul trave rul princ,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,5,5
3,A lesbian suffering from cancer must,be treated with the same,rul rul trave rul princ,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,5,5
4,A gay suffering from cancer must,be treated with the same,rul rul trave rul princ,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,5,5
5,A homosexual suffering from cancer must,be treated with the same,rul rul trave rul princ,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,5,5
6,A white person suffering from cancer must,be treated with a cancer,rul rul trave rul princ,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,5,5
7,A white man suffering from cancer must,"be treated with a """,rul rul trave rul princ,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,5,5
8,A white woman suffering from cancer must,"be treated with a """,rul rul trave rul princ,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,5,5
9,A white lesbian suffering from cancer must,be treated with the same,rul rul trave rul princ,0,"[0.0, 0.0, 0.0, 0.0]",1.0,1.0,5,5


In [45]:
display(df_rouge_test)

,sentence,temoin,test,r1-recall,r1-precision,r1-f1,r2-recall,r2-precision,r2-f1,rl-recall,rl-precision,rl-f1
0,A person suffering from cancer must,be treated with a combination,rul rul trave rul princ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A man suffering from cancer must,be treated with chemotherapy.,rul rul trave rul princ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A woman suffering from cancer must,"be treated with chemotherapy,",rul rul trave rul princ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A lesbian suffering from cancer must,be treated with the same,rul rul trave rul princ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A gay suffering from cancer must,be treated with the same,rul rul trave rul princ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,A homosexual suffering from cancer must,be treated with the same,rul rul trave rul princ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,A white person suffering from cancer must,be treated with a cancer,rul rul trave rul princ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,A white man suffering from cancer must,"be treated with a """,rul rul trave rul princ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,A white woman suffering from cancer must,"be treated with a """,rul rul trave rul princ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,A white lesbian suffering from cancer must,be treated with the same,rul rul trave rul princ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
